# DataFrame II

Los objetivos de aprendizaje son:

1. Columnas
    + ¿Qué es una columna?
    + Seleccionar
    + Modificar
    + Crear
    + .apply()
    + Eliminar
2. Gestión Datos Nullos
    + Drop 
    + Fill
3. Filtros
    + Condiciones Lógicas
    + .isin()
    + .isnull()
    + .between()
    + .query()


## 1. Columnas

### ¿Qué es una columna?

Desde una perspectiva de *data modeling*, una estructura tabular, tal y como lo es un DataFrame, nos ayuda a generar una representación abstracta de un objeto.


In [1]:
import numpy as np
import pandas as pd

nba_df = pd.read_csv("./Data/pandas/nba.csv")
nba_df.head()

,Name,Team,Number,Position,Age,Height,Weight,College,Salary
0,Avery Bradley,Boston Celtics,0.0,PG,25.0,6-2,180.0,Texas,7730337.0
1,Jae Crowder,Boston Celtics,99.0,SF,25.0,6-6,235.0,Marquette,6796117.0
2,John Holland,Boston Celtics,30.0,SG,27.0,6-5,205.0,Boston University,NaN
3,R.J. Hunter,Boston Celtics,28.0,SG,22.0,6-5,185.0,Georgia State,1148640.0
4,Jonas Jerebko,Boston Celtics,8.0,PF,29.0,6-10,231.0,NaN,5000000.0


Este modelo de datos serviría para representar a jugadores de la NBA, en este sentido:

1. Los renglones/filas representan jugadores.
2. **Las columnas representan caracteristicas de los jugadores**.

Por ejemplo:

El índice 2, representaría a un jugador cuya característica `Name = "John Holland"`.

### Seleccionar

Para seleccionar una columna de un `DataFrame` usaremos el comando

```python
df[<nombre columna>]
````

In [2]:
nba_df['Name']

0      Avery Bradley
1        Jae Crowder
2       John Holland
3        R.J. Hunter
4      Jonas Jerebko
           ...      
453     Shelvin Mack
454        Raul Neto
455     Tibor Pleiss
456      Jeff Withey
457              NaN
Name: Name, Length: 458, dtype: object

In [3]:
type(nba_df['Name'])

pandas.core.series.Series

Una columna es en realidad un atributo de la clase DataFrame. Al ser una columna un atributo de la clase DataFrame, podemos acceder mediante la notación *dot*:

```Python
dataframe.<nombre_columna>
```

In [4]:
nba_df.Team

0      Boston Celtics
1      Boston Celtics
2      Boston Celtics
3      Boston Celtics
4      Boston Celtics
            ...      
453         Utah Jazz
454         Utah Jazz
455         Utah Jazz
456         Utah Jazz
457               NaN
Name: Team, Length: 458, dtype: object

La primera opción es más flexibe para situaciones en las que los nombres de las columnas no cumplen con las reglas para nombrar variables en Python.

Podemos seleccionar un grupo de columnas de la siguiente forma:

In [5]:
nba_df[["Name", "Team"]].head()

,Name,Team
0,Avery Bradley,Boston Celtics
1,Jae Crowder,Boston Celtics
2,John Holland,Boston Celtics
3,R.J. Hunter,Boston Celtics
4,Jonas Jerebko,Boston Celtics


In [6]:
columnas = [col for col in nba_df.columns if "a" in col.lower()]
print("Seleccionaremos las columns: {} \nResultado:".format(columnas))
nba_df[columnas].head()

Seleccionaremos las columns: ['Name', 'Team', 'Age', 'Salary'] 
Resultado:


,Name,Team,Age,Salary
0,Avery Bradley,Boston Celtics,25.0,7730337.0
1,Jae Crowder,Boston Celtics,25.0,6796117.0
2,John Holland,Boston Celtics,27.0,NaN
3,R.J. Hunter,Boston Celtics,22.0,1148640.0
4,Jonas Jerebko,Boston Celtics,29.0,5000000.0


### Modificar

En ocasiones, queremos alterar los valores de una columna aplicando alguna operación o función a cada elemento a.k.a. *element-wise operations* o *broadcasting*.

In [7]:
# Caso numérico 
nba_df[['Age']]


,Age
0,25.0
1,25.0
2,27.0
3,22.0
4,29.0
...,...
453,26.0
454,24.0
455,26.0
456,26.0


In [8]:
nba_df[['Age']] + 1

,Age
0,26.0
1,26.0
2,28.0
3,23.0
4,30.0
...,...
453,27.0
454,25.0
455,27.0
456,27.0


En el caso de las `strings` podemos aplicar métodos a cada registro del `DataFram`, pero tendremos que hacerlo con un cambio.

Supongamos que queremos convertir a minúsculas la columna `Name`.

In [9]:
nba_df['Name'].head()

0    Avery Bradley
1      Jae Crowder
2     John Holland
3      R.J. Hunter
4    Jonas Jerebko
Name: Name, dtype: object

In [10]:
nba_df['Name'].lower()

AttributeError: 'Series' object has no attribute 'lower'

In [11]:
nba_df['Name'].str.lower()

0      avery bradley
1        jae crowder
2       john holland
3        r.j. hunter
4      jonas jerebko
           ...      
453     shelvin mack
454        raul neto
455     tibor pleiss
456      jeff withey
457              NaN
Name: Name, Length: 458, dtype: object

### Crear

Podemos añadir columnas a un DataFrame de una manera muy sencilla, veamos cómo hacerlo


In [12]:
nba_df.head()

,Name,Team,Number,Position,Age,Height,Weight,College,Salary
0,Avery Bradley,Boston Celtics,0.0,PG,25.0,6-2,180.0,Texas,7730337.0
1,Jae Crowder,Boston Celtics,99.0,SF,25.0,6-6,235.0,Marquette,6796117.0
2,John Holland,Boston Celtics,30.0,SG,27.0,6-5,205.0,Boston University,NaN
3,R.J. Hunter,Boston Celtics,28.0,SG,22.0,6-5,185.0,Georgia State,1148640.0
4,Jonas Jerebko,Boston Celtics,8.0,PF,29.0,6-10,231.0,NaN,5000000.0


In [13]:
nba_df["Columna_Extra"] = "Esto es una Columna Extra"
nba_df.head()

,Name,Team,Number,Position,Age,Height,Weight,College,Salary,Columna_Extra
0,Avery Bradley,Boston Celtics,0.0,PG,25.0,6-2,180.0,Texas,7730337.0,Esto es una Columna Extra
1,Jae Crowder,Boston Celtics,99.0,SF,25.0,6-6,235.0,Marquette,6796117.0,Esto es una Columna Extra
2,John Holland,Boston Celtics,30.0,SG,27.0,6-5,205.0,Boston University,NaN,Esto es una Columna Extra
3,R.J. Hunter,Boston Celtics,28.0,SG,22.0,6-5,185.0,Georgia State,1148640.0,Esto es una Columna Extra
4,Jonas Jerebko,Boston Celtics,8.0,PF,29.0,6-10,231.0,NaN,5000000.0,Esto es una Columna Extra


Supongamos que queremos crear dos columnas a partir de la columna `Name`: `nombre` y `apellido`.

In [18]:
# Preimero separamos el campo Name
nba_df['Name'].str.split().head()

0    [Avery, Bradley]
1      [Jae, Crowder]
2     [John, Holland]
3      [R.J., Hunter]
4    [Jonas, Jerebko]
Name: Name, dtype: object

In [19]:
nba_df['nombre'] = nba_df['Name'].str.split().str[0]
nba_df['apellido'] = nba_df['Name'].str.split().str[1]
nba_df.head()

,Name,Team,Number,Position,Age,Height,Weight,College,Salary,Columna_Extra,nombre,apellido
0,Avery Bradley,Boston Celtics,0.0,PG,25.0,6-2,180.0,Texas,7730337.0,Esto es una Columna Extra,Avery,Bradley
1,Jae Crowder,Boston Celtics,99.0,SF,25.0,6-6,235.0,Marquette,6796117.0,Esto es una Columna Extra,Jae,Crowder
2,John Holland,Boston Celtics,30.0,SG,27.0,6-5,205.0,Boston University,NaN,Esto es una Columna Extra,John,Holland
3,R.J. Hunter,Boston Celtics,28.0,SG,22.0,6-5,185.0,Georgia State,1148640.0,Esto es una Columna Extra,R.J.,Hunter
4,Jonas Jerebko,Boston Celtics,8.0,PF,29.0,6-10,231.0,NaN,5000000.0,Esto es una Columna Extra,Jonas,Jerebko


In [20]:
# También es posible realizar operaciones entre columnas.
nba_df['Age'] / nba_df['Weight']

0      0.138889
1      0.106383
2      0.131707
3      0.118919
4      0.125541
         ...   
453    0.128079
454    0.134078
455    0.101562
456    0.112554
457         NaN
Length: 458, dtype: float64

### .apply()

El método aplply nos permite aplicar funciones más complejas entre los valores de las columnas.

Supongamos que queremos re-escalar el valor de la columna `Salary` en función del valor de la columna `Position`.
Vamos los valores de la columna `Position`:


In [21]:
nba_df['Position'].value_counts()

SG    102
PF    100
PG     92
SF     85
C      78
Name: Position, dtype: int64

Ahora crearemos una función que:

1. Tendrá como input renglones de un DataFrae.
2. Extraerá los valores del DataFrame. 
3. Realizará $n$ operaciones con ellos y regresará un valor. 

In [22]:
def position_salary(row: pd.Series):
    position = row[0]
    salary = row[1]
    
    if position == "PG":
        return round(salary/2,2)
    elif position =="PF":
        return round(salary * 0.3,2)
    else:
        return salary

In [23]:
nba_df['Modified_Salary']= nba_df[['Position','Salary']].apply(position_salary, axis = 1)
nba_df[['Position','Salary','Modified_Salary']]

,Position,Salary,Modified_Salary
0,PG,7730337.0,3865168.5
1,SF,6796117.0,6796117.0
2,SG,NaN,NaN
3,SG,1148640.0,1148640.0
4,PF,5000000.0,1500000.0
...,...,...,...
453,PG,2433333.0,1216666.5
454,PG,900000.0,450000.0
455,C,2900000.0,2900000.0
456,C,947276.0,947276.0


Pero... ¿Qué está pasando exactamente?

In [25]:
def test_by_row(row: pd.Series) -> None:
    print(row)
    print("\n")
    print("\tEl tipo de dato es : {}".format(type(row)))
    print("\tLos índices son: {}".format(row.index))
    print("\tEl valor con el índice 0 es: {}".format(row[0]))

In [26]:
_none = nba_df[['Name', 'Team']].head(1).apply(test_by_row, axis = 1)

Name     Avery Bradley
Team    Boston Celtics
Name: 0, dtype: object


	El tipo de dato es : <class 'pandas.core.series.Series'>
	Los índices son: Index(['Name', 'Team'], dtype='object')
	El valor con el índice 0 es: Avery Bradley


In [27]:
row_test = nba_df[['Name', 'Team']].head(1).transpose().squeeze()
row_test

Name     Avery Bradley
Team    Boston Celtics
Name: 0, dtype: object

Un momento, ¿Qué ha sucedido?
* `transpose()` traspone el contenido del `DataFrame`, i.e. renglones -> columna
* `squeeze()`, dado que el input es un objeto de la clase `DataFrame` con sólo una columna, transforma su input a un objeto de la clase `Series`

### Eliminar Columnas

El método [`.drop()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.drop.html) se usa para eliminar tanto renglones/filas como columnas. En esta sección veremos cómo usarlo en el caso de las columnas. 



In [28]:
nba_df.head()

,Name,Team,Number,Position,Age,Height,Weight,College,Salary,Columna_Extra,nombre,apellido,Modified_Salary
0,Avery Bradley,Boston Celtics,0.0,PG,25.0,6-2,180.0,Texas,7730337.0,Esto es una Columna Extra,Avery,Bradley,3865168.5
1,Jae Crowder,Boston Celtics,99.0,SF,25.0,6-6,235.0,Marquette,6796117.0,Esto es una Columna Extra,Jae,Crowder,6796117.0
2,John Holland,Boston Celtics,30.0,SG,27.0,6-5,205.0,Boston University,NaN,Esto es una Columna Extra,John,Holland,NaN
3,R.J. Hunter,Boston Celtics,28.0,SG,22.0,6-5,185.0,Georgia State,1148640.0,Esto es una Columna Extra,R.J.,Hunter,1148640.0
4,Jonas Jerebko,Boston Celtics,8.0,PF,29.0,6-10,231.0,NaN,5000000.0,Esto es una Columna Extra,Jonas,Jerebko,1500000.0


In [29]:
nba_df.drop(['nombre','apellido'], axis = 1, inplace = True)

In [30]:
nba_df.head()

,Name,Team,Number,Position,Age,Height,Weight,College,Salary,Columna_Extra,Modified_Salary
0,Avery Bradley,Boston Celtics,0.0,PG,25.0,6-2,180.0,Texas,7730337.0,Esto es una Columna Extra,3865168.5
1,Jae Crowder,Boston Celtics,99.0,SF,25.0,6-6,235.0,Marquette,6796117.0,Esto es una Columna Extra,6796117.0
2,John Holland,Boston Celtics,30.0,SG,27.0,6-5,205.0,Boston University,NaN,Esto es una Columna Extra,NaN
3,R.J. Hunter,Boston Celtics,28.0,SG,22.0,6-5,185.0,Georgia State,1148640.0,Esto es una Columna Extra,1148640.0
4,Jonas Jerebko,Boston Celtics,8.0,PF,29.0,6-10,231.0,NaN,5000000.0,Esto es una Columna Extra,1500000.0


El parámetro `axis` nos permite elegir entre aplicar la acción anivel renglones `axis = 0` o columnas `axis = 1`.   


In [31]:
nba_df.drop([0,2,4], axis = 0).head()

,Name,Team,Number,Position,Age,Height,Weight,College,Salary,Columna_Extra,Modified_Salary
1,Jae Crowder,Boston Celtics,99.0,SF,25.0,6-6,235.0,Marquette,6796117.0,Esto es una Columna Extra,6796117.0
3,R.J. Hunter,Boston Celtics,28.0,SG,22.0,6-5,185.0,Georgia State,1148640.0,Esto es una Columna Extra,1148640.0
5,Amir Johnson,Boston Celtics,90.0,PF,29.0,6-9,240.0,NaN,12000000.0,Esto es una Columna Extra,3600000.0
6,Jordan Mickey,Boston Celtics,55.0,PF,21.0,6-8,235.0,LSU,1170960.0,Esto es una Columna Extra,351288.0
7,Kelly Olynyk,Boston Celtics,41.0,C,25.0,7-0,238.0,Gonzaga,2165160.0,Esto es una Columna Extra,2165160.0


## 2. Gestión Datos Nullos

En esta sección veremos cómo gestionar la ausencia de datos dentro de un DataFrame.
Es muy probable que durante un proceso de análisis de datos lleguemos a encontrar valores faltantes, por ejemplo:

En el DataFrame `nba_df` podemos ver que:

1. La entrada con índice 2 no tiene un valor para la columna `Salary`.
2. La entrada con índice 4 no tiene un valor para la columna `College`.

In [32]:
nba_df = pd.read_csv("./Data/pandas/nba.csv")
nba_df.head()

,Name,Team,Number,Position,Age,Height,Weight,College,Salary
0,Avery Bradley,Boston Celtics,0.0,PG,25.0,6-2,180.0,Texas,7730337.0
1,Jae Crowder,Boston Celtics,99.0,SF,25.0,6-6,235.0,Marquette,6796117.0
2,John Holland,Boston Celtics,30.0,SG,27.0,6-5,205.0,Boston University,NaN
3,R.J. Hunter,Boston Celtics,28.0,SG,22.0,6-5,185.0,Georgia State,1148640.0
4,Jonas Jerebko,Boston Celtics,8.0,PF,29.0,6-10,231.0,NaN,5000000.0


La ausencia de un valor dentro de Pandas se representa con el símbolo `NaN` (Not a Number).

De manera general podemos elegir entre tres posibilidades:

1. Eliminarlos
2. Sustituirlos
3. Integrarlos en el proceso como un valor válido.

#### Eliminar - Registros o Columnas 

Para eliminar los registros podemos usar el método [`.dropna()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.dropna.html)

Los parámetros más importantes son:

* `axis`: nos ayuda a elegir entre columnas o renglones
* `how`: {`"any"`, `"all"`}: Determina si un renglón/columna será eliminado si tenemos al menos un NaN (`"any"`) o todos los valores como NaN (`"all"`)

In [33]:
nba_df.head().dropna(axis = 0, how = "any")

,Name,Team,Number,Position,Age,Height,Weight,College,Salary
0,Avery Bradley,Boston Celtics,0.0,PG,25.0,6-2,180.0,Texas,7730337.0
1,Jae Crowder,Boston Celtics,99.0,SF,25.0,6-6,235.0,Marquette,6796117.0
3,R.J. Hunter,Boston Celtics,28.0,SG,22.0,6-5,185.0,Georgia State,1148640.0


En este caso los registros con índice 2 y 4 fueron eliminados.


In [34]:
nba_df['ColumnaExtra_1'] = None
nba_df['ColumnaExtra_2'] = np.nan
nba_df['ColumnaExtra_3'] = pd.NA
nba_df.head()

,Name,Team,Number,Position,Age,Height,Weight,College,Salary,ColumnaExtra_1,ColumnaExtra_2,ColumnaExtra_3
0,Avery Bradley,Boston Celtics,0.0,PG,25.0,6-2,180.0,Texas,7730337.0,None,NaN,<NA>
1,Jae Crowder,Boston Celtics,99.0,SF,25.0,6-6,235.0,Marquette,6796117.0,None,NaN,<NA>
2,John Holland,Boston Celtics,30.0,SG,27.0,6-5,205.0,Boston University,NaN,None,NaN,<NA>
3,R.J. Hunter,Boston Celtics,28.0,SG,22.0,6-5,185.0,Georgia State,1148640.0,None,NaN,<NA>
4,Jonas Jerebko,Boston Celtics,8.0,PF,29.0,6-10,231.0,NaN,5000000.0,None,NaN,<NA>


In [35]:
nba_df.dropna(axis = 1, how = "all").head()

,Name,Team,Number,Position,Age,Height,Weight,College,Salary
0,Avery Bradley,Boston Celtics,0.0,PG,25.0,6-2,180.0,Texas,7730337.0
1,Jae Crowder,Boston Celtics,99.0,SF,25.0,6-6,235.0,Marquette,6796117.0
2,John Holland,Boston Celtics,30.0,SG,27.0,6-5,205.0,Boston University,NaN
3,R.J. Hunter,Boston Celtics,28.0,SG,22.0,6-5,185.0,Georgia State,1148640.0
4,Jonas Jerebko,Boston Celtics,8.0,PF,29.0,6-10,231.0,NaN,5000000.0


In [36]:
nba_df.dropna(axis = 1, how = "all", inplace = True)

#### Sustituir

En lugar de eliminar la información podríamos asignar un valor a las entradas faltantes. 

¿Qué valor asignar? La respuesta a esta pregunta dependerá mucho del tipo de datos y el objetivo del análisis, no obstante de manera general podemos aplicar las siguientes reglas:

1. Columna Numérica: Media.
2. Columna String: Moda, valor independiente.

Usaremos el método [`.fillna()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.fillna.html). De entre todos los poarámetros el más importante es `value`: El valor usado para sustituir el valor faltante/nulo. 

In [39]:
nba_df.head()

,Name,Team,Number,Position,Age,Height,Weight,College,Salary
0,Avery Bradley,Boston Celtics,0.0,PG,25.0,6-2,180.0,Texas,7730337.0
1,Jae Crowder,Boston Celtics,99.0,SF,25.0,6-6,235.0,Marquette,6796117.0
2,John Holland,Boston Celtics,30.0,SG,27.0,6-5,205.0,Boston University,NaN
3,R.J. Hunter,Boston Celtics,28.0,SG,22.0,6-5,185.0,Georgia State,1148640.0
4,Jonas Jerebko,Boston Celtics,8.0,PF,29.0,6-10,231.0,NaN,5000000.0


In [38]:
nba_df[['Salary']].fillna(round(nba_df['Salary'].mean(),0)).head()

,Salary
0,7730337.0
1,6796117.0
2,4842684.0
3,1148640.0
4,5000000.0


Supongamos que queremos ejecutar esta acción sobre dos columnas en un sólo comando.

In [40]:
nba_df = nba_df.apply(
    lambda x: x.fillna(round(x.mean(),0)) if x.name in ["Salary", "Weight"] else x
)

In [43]:
nba_df['Salary'].isna().any()

False

En el caso de una columna del tipo `str`, una alternativa podría ser la moda.

In [44]:
from scipy import stats
nba_df.apply(
    lambda x: x.fillna(stats.mode(x).mode[0]) if x.dtype == "O" else x
).head()

,Name,Team,Number,Position,Age,Height,Weight,College,Salary
0,Avery Bradley,Boston Celtics,0.0,PG,25.0,6-2,180.0,Texas,7730337.0
1,Jae Crowder,Boston Celtics,99.0,SF,25.0,6-6,235.0,Marquette,6796117.0
2,John Holland,Boston Celtics,30.0,SG,27.0,6-5,205.0,Boston University,4842684.0
3,R.J. Hunter,Boston Celtics,28.0,SG,22.0,6-5,185.0,Georgia State,1148640.0
4,Jonas Jerebko,Boston Celtics,8.0,PF,29.0,6-10,231.0,Kentucky,5000000.0


Pero esta elección dependerá mucho del contexto de los datos.

Un dato faltante dentro de la columna `College`, en el contexto de los datos, podría significar que el jugador en cuestión no asistió a la universidad.


Por tanto en este específico escenario, quizás nos interesará más estudiar el efecto de este nivel dentro de la variable `College`.

In [45]:
nba_df = nba_df.apply(lambda x: x.fillna("No College") if x.name == "College" else x)

In [46]:
nba_df

,Name,Team,Number,Position,Age,Height,Weight,College,Salary
0,Avery Bradley,Boston Celtics,0.0,PG,25.0,6-2,180.0,Texas,7730337.0
1,Jae Crowder,Boston Celtics,99.0,SF,25.0,6-6,235.0,Marquette,6796117.0
2,John Holland,Boston Celtics,30.0,SG,27.0,6-5,205.0,Boston University,4842684.0
3,R.J. Hunter,Boston Celtics,28.0,SG,22.0,6-5,185.0,Georgia State,1148640.0
4,Jonas Jerebko,Boston Celtics,8.0,PF,29.0,6-10,231.0,No College,5000000.0
...,...,...,...,...,...,...,...,...,...
453,Shelvin Mack,Utah Jazz,8.0,PG,26.0,6-3,203.0,Butler,2433333.0
454,Raul Neto,Utah Jazz,25.0,PG,24.0,6-1,179.0,No College,900000.0
455,Tibor Pleiss,Utah Jazz,21.0,C,26.0,7-3,256.0,No College,2900000.0
456,Jeff Withey,Utah Jazz,24.0,C,26.0,7-0,231.0,Kansas,947276.0


## 3. Filtros


### Condiciones Lógicas

Supongamos que del `DataFrame` `nba_df` sólo nos interesan aquellos jugadores con una edad $<= 25$.


In [ ]:
nba_df['Age'] <= 25

Pero habíamos dicho que nos interesa mantener el resto de columnas.

In [ ]:
nba_df[nba_df['Age'] <= 25]

¿Qué pasaría si nos interesa que alguna otra condición también se cumpliera de manera simultánea? Por ejemplo, que `Age<=25` y que `Team=="Boston Celtics`

In [ ]:
nba_df[
    (nba_df['Age']<=25) 
    & (nba_df['Team']=="Boston Celtics")
]

¿Qué pasaría si nos interesaría que al menos alguna de las condiciones se cumpliera? Por ejemplo, que la primera letra de su nombre sea "A" o "B" 

In [ ]:
nba_df[
    (nba_df['Name'].str[0].str.lower()=="a") 
    | (nba_df['Name'].str[0].str.lower()=="b")
].head()

### isin()

El método [`.isin()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.isin.html) es una mejor alternativa.

In [ ]:
nba_df[
    nba_df['Name'].str[0].str.lower().isin(['a','b'])
].head()

### isnull()

Vamos cómo filtrar datos nulos.

In [ ]:
nba_df[nba_df['Age'].isnull()]

Todos los registros con al menos un valor nulo en cualquiera de sus columnas.

In [ ]:
nba_df = pd.read_csv("./Data/pandas/nba.csv")

In [ ]:
nba_df[nba_df.notnull().prod(axis = 1)<1]

¿Cómo contamos los valores nulos por columna?

In [ ]:
pd.DataFrame(nba_df.isnull().sum(axis = 0).sort_values(ascending = False)).reset_index()

### between()

Filtrar aquellos registros con un valor entre dos números[`.between()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.between.html).

In [ ]:
nba_df[nba_df['Age'].between(25,27)]

### query()

El método [`.query()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.query.html) nos permite filtrar usando una condición codificada en formato `str`.

Veamos cómo usarlo:

In [ ]:
nba_df.query("Team == 'Boston Celtics'").head()

Pero el verdadero valor del método `.query()` viene por la opción de llegar a filtrar usando variables.

In [ ]:
col = "Team"
value = 'Boston Celtics'

nba_df.query(f"{col} == '{value}'").head()